Note: following these instructions require permissions to sudo as `analytics-product`, which owns the `canonical_data` database.

In [1]:
import pandas as pd

import wmfdata as wmf

You are using Wmfdata v2.0.1, but v2.2.0 is available.

To update, run `pip install --upgrade git+https://github.com/wikimedia/wmfdata-python.git@release`.

To see the changes, refer to https://github.com/wikimedia/wmfdata-python/blob/release/CHANGELOG.md.


Start by loading the dataset into your own database.

In [3]:
MY_DATABASE = "nshahquinn"

wmf.hive.load_csv(
    "wikis.tsv",
    field_spec="""
        database_code   STRING  COMMENT 'Same as wiki_db in MediaWiki History, e.g. enwiki', 
        domain_name     STRING  COMMENT 'e.g. en.wikipedia.org', 
        database_group  STRING  COMMENT 'e.g. wikipedia',
        language_code   STRING  COMMENT 'e.g. en',
        mobile_domain_name STRING COMMENT 'e.g. en.m.wikipedia.org',
        language_name   STRING  COMMENT 'e.g. English',
        status          STRING  COMMENT 'open/closed',
        visibility      STRING  COMMENT 'public/private',
        editability     STRING  COMMENT 'public/private',
        english_name    STRING  COMMENT 'e.g. English Wikipedia'
    """,
    db_name=MY_DATABASE,
    table_name="wikis",
    sep="\t"
)

/home/nshahquinn-wmf/.conda/envs/2023-10-28T00.57.48_nshahquinn-wmf/lib/python3.10/site-packages/wmfdata/hive.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  response = pd.read_sql(command, conn)


Now, since it's not possible to run `sudo` commands in our Jupyter environment, in a plain SSH connection, run the following command:

In [3]:
print(
    "sudo -u analytics-product kerberos-run-command analytics-product "
    "hive -e 'DROP TABLE IF EXISTS canonical_data.wikis; "
    f"CREATE TABLE canonical_data.wikis AS SELECT * FROM {MY_DATABASE}.wikis'"
)

sudo -u analytics-product kerberos-run-command analytics-product hive -e 'DROP TABLE IF EXISTS canonical_data.wikis; CREATE TABLE canonical_data.wikis AS SELECT * FROM nshahquinn.wikis'


Once that has completed, verify that the newly loaded data matches our local copy.

In [4]:
loaded = wmf.presto.run("""
    SELECT *
    FROM canonical_data.wikis
""")

local = pd.read_csv(
    "wikis.tsv", 
    sep="\t",
    # By default, Pandas will parse the string "nan" as a null. However, we have
    # several wikis in the Southern Min language, which has the code "nan". This
    # prevents Pandas from interpreting *any* value as null.
    keep_default_na = False,
    # We still want to be able to represent null values in the TSV. By default, 
    # Pandas writes nulls as empty fields, so we ask Pandas to interpret
    # empty strings (and nothing else) as nulls.
    na_values = [""]
)

assert local.equals(loaded)